In [1]:
!pip install transformers

     |████████████████████████████████| 9.0 MB 83 kB/s eta 0:00:014
     |████████████████████████████████| 388 kB 345 kB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 256 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 162 kB/s eta 0:00:01
     |████████████████████████████████| 171 kB 136 kB/s eta 0:00:01


In [ ]:
import torch
from transformers import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x=='positive' else 0)
df.head()

In [ ]:
#dataset dimensions
df.shape

In [ ]:
#Labeled reviews barplot
sns.countplot(df.sentiment)
plt.xlabel('sentiments')

The Transformers library provides a wide variety of Transformer models (including BERT). It works with TensorFlow and PyTorch! It also includes prebuild tokenizers to do the heavy work required for bert model input.

In [ ]:
#Selecting the bert-base-cased
PRE_TRAINED_MODEL_NAME = '../input/bert-base-cased/'

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
#Let's see how Bertencoder is encoding a sentence
sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

In [ ]:
tokenizer.sep_token, tokenizer.sep_token_id

In [ ]:
tokenizer.cls_token, tokenizer.cls_token_id

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

In [ ]:
tokenizer.unk_token, tokenizer.unk_token_id

In [ ]:
#To do all preprocessing you specify some parameters in the encod_plus() method of the tokenizer
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

In [ ]:
#The tokens ids list
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

In [ ]:
#The attentions masked tokens
print(len(encoding['attention_mask'][0]))
encoding['attention_mask']

In [ ]:
#let's see how the sentence is tokenized with bert tokenizer
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

In [ ]:
token_lens = []

for txt in df.review:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))
sns.distplot(token_lens)
plt.xlim([0, 500]);
plt.xlabel('Token count')

In [ ]:
MAX_LEN = 200      #for not consuming much resources
RANDOM_SEED = 42
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [ ]:
#doing the split of the dataset into training, validation and testing sets
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)


In [ ]:
df_train.shape, df_val.shape, df_test.shape